In [85]:
import pymongo
import numpy
import json
from datetime import *

In [2]:
mc = pymongo.MongoClient()
db = mc['allrecipes']
recipes_coll = db['recipes']
results_coll = db['search_results']

In [3]:
len(list(recipes_coll.find()))

290

In [4]:
len(list(results_coll.find()))

488

In [16]:
cursor = recipes_coll.find()
repeats = set()
for recipe in cursor:
    if len(list(recipes_coll.find({'id':recipe['id']}))) > 1:
        repeats.add(recipe['id'])
print(repeats)

set()


In [13]:
list(recipes_coll.find({'id':15914}))

[{'_id': ObjectId('5ca3cf24c6d0b005892ceb08'),
  'id': 15914,
  'name': 'Chili Soup',
  'href': 'https://www.allrecipes.com/recipe/15914/chili-soup/',
  'category': {'lvl_1': 'Soups, Stews and Chili',
   'lvl_2': 'Chili',
   'lvl_3': 'Beef Chili'},
  'rating_info': {'rating': 4.4642858505249, 'made_by': 48, 'reviews': 25},
  'submitter_info': None,
  'ingredients': [{'quantity': 3.0,
    'units': 'pound',
    'ingredient': 'ground beef'},
   {'quantity': 1.5, 'units': 'cup', 'ingredient': 'onion'},
   {'quantity': 2.0, 'units': 'each', 'ingredient': 'potatoes'},
   {'quantity': 60.0, 'units': 'ounce', 'ingredient': 'kidney beans'},
   {'quantity': 43.0, 'units': 'ounce', 'ingredient': 'tomato soup'},
   {'quantity': 8.0, 'units': 'cup', 'ingredient': 'tomato juice'},
   {'quantity': 3.0, 'units': 'teaspoon', 'ingredient': 'chili powder'},
   {'quantity': 8.0, 'units': 'cup', 'ingredient': 'water'}],
  'directions': {'timing': None,
   'steps': ['In a large pot over medium heat, combine

In [15]:
for recipe_id in repeats:
    recipes_coll.delete_one({'id':recipe_id})

In [32]:
cursor = results_coll.find()
id_list = []
id_set = set()
repeats = set()
for result in cursor:
    if len(list(results_coll.find({'id':result['id']}))) > 1:
        repeats.add(result['id'])
    id_list.append(result['id'])
    id_set.add(result['id'])
print('List length :', len(id_list))
print('Set length  :', len(id_set))
print('Repeats     :', len(repeats))

List length : 386
Set length  : 386
Repeats     : 0


In [31]:
len_prev = 0
while True:
    cursor = results_coll.find()
    repeats = set()
    len_current = 0
    for result in cursor:
        if len(list(results_coll.find({'id':result['id']}))) > 1:
            repeats.add(result['id'])
        len_current += 1
    print('Len. prev.:', len_prev)
    print('Len. curr.:', len_current)
    if len(repeats) == 0:
        break
    if len_prev == len_current:
        for result_id in repeats:
            results_coll.delete_one({'id':result_id, 'viewed':1})
    else:
        for result_id in repeats:
            results_coll.delete_one({'id':result_id, 'viewed':0})
    len_prev = len_current

Len. prev.: 0
Len. curr.: 486
Len. prev.: 486
Len. curr.: 426
Len. prev.: 426
Len. curr.: 387
Len. prev.: 387
Len. curr.: 387
Len. prev.: 387
Len. curr.: 386


In [29]:
results_coll.find_one()

{'_id': ObjectId('5ca453368fd54bc4290e015a'),
 'id': 228293,
 'name': 'Curry Stand Chicken Tikka Masala Sauce',
 'href': 'https://www.allrecipes.com/recipe/228293/curry-stand-chicken-tikka-masala-sauce',
 'viewed': 1}

In [33]:
mc.list_database_names()

['admin',
 'allrecipes',
 'config',
 'fraud',
 'fraud_case_study',
 'local',
 'nyt_dump',
 'recipes_test']

In [34]:
db_test = mc['recipes_test']

In [35]:
db_test.list_collection_names()

['search_results', 'recipes']

In [36]:
recipes_test = db_test['recipes']

In [37]:
recipes_test.find_one()

{'_id': ObjectId('5ca3adba8fd54b1b485da598'),
 'id': '78299',
 'name': 'Boilermaker Tailgate Chili',
 'href': 'https://www.allrecipes.com/recipe/78299/boilermaker-tailgate-chili/',
 'category': {'lvl_1': 'Soups, Stews and Chili',
  'lvl_2': 'Chili',
  'lvl_3': 'Pork Chili'},
 'rating_info': {'rating': 4.8183069229126, 'made_by': 12000, 'reviews': 4000},
 'submitter_info': {'id': 591623,
  'name': 'MIGHTYPURDUE22',
  'followers': 41,
  'href': 'https://www.allrecipes.com/cook/591623/'},
 'ingredients': [{'quantity': 2.0,
   'units': 'pound',
   'ingredient': 'ground beef chuck'},
  {'quantity': 1.0, 'units': 'pound', 'ingredient': 'italian sausage'},
  {'quantity': 45.0, 'units': 'ounce', 'ingredient': 'chili beans'},
  {'quantity': 15.0,
   'units': 'ounce',
   'ingredient': 'chili beans in spicy sauce'},
  {'quantity': 56.0, 'units': 'ounce', 'ingredient': 'tomatoes with juice'},
  {'quantity': 6.0, 'units': 'ounce', 'ingredient': 'tomato paste'},
  {'quantity': 1.0, 'units': 'each', 

In [38]:
fake = {'id':12345, 'name':'My Fake Recipe', 'ingredients':None}
recipes_test.insert_one(fake)

In [100]:
cursor = recipes_test.find().sort([('_id',-1)]).limit(1)
recipe = next(cursor)

In [73]:
timestamp = recipe['_id'].generation_time

In [76]:
timestamp.strftime('%Y-%m-%d %H:%M:%S')

'2019-04-03 19:10:11'

In [86]:
now = datetime.now(timezone.utc)

In [88]:
diff = now-timestamp

In [92]:
str(diff)

'0:24:21.896417'

In [99]:
bool(int('0'))

False

In [101]:
def get_next_recipe(collection, reverse=False):
    direction = -1 if reverse else 1
    cursor = collection.find({'viewed':0}).sort([('_id',direction)]).limit(1)
    return next(cursor)

In [102]:
get_next_recipe(recipes_coll)

StopIteration: 

In [111]:
cursor = results_coll.find({'viewed':0}).sort([('_id',1)]).limit(2)

In [112]:
next(cursor)

{'_id': ObjectId('5ca3f1e0c6d0b007d9d1a215'),
 'id': 244979,
 'name': "Chef John's Peanut Curry Chicken",
 'href': 'https://www.allrecipes.com/recipe/244979/chef-johns-peanut-curry-chicken',
 'viewed': 0}